In [203]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV

In [204]:
data = pd.read_csv('C:\Th-th-ch-BAI-Labs\Bai_5\data\insurance_claims.csv')

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Admin\AppData\Local\Temp\ipykernel_20844\1517680743.py:1: SyntaxWarning: invalid escape sequence '\T'
  data = pd.read_csv('C:\Th-th-ch-BAI-Labs\Bai_5\data\insurance_claims.csv')


In [205]:
dropc_list = ['policy_number','policy_bind_date','policy_state','insured_zip','incident_location','incident_date',
           'incident_state','incident_city','insured_hobbies','auto_make','auto_model','auto_year', '_c39',
           'age','total_claim_amount']
dropr_list = [578]

In [206]:
# don't remove
df = data.copy()
df.drop(columns=dropc_list, inplace=True)
df.drop(index=dropr_list, inplace=True)
df['fraud_reported'] = df['fraud_reported'].replace({'Y':1, 'N':0})
target = df.pop('fraud_reported')
df.replace('?', np.nan, inplace = True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_20844\2222906449.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['fraud_reported'] = df['fraud_reported'].replace({'Y':1, 'N':0})


In [207]:
df['collision_type'] = df['collision_type'].fillna(df['collision_type'].mode()[0])
df['property_damage'] = df['property_damage'].fillna(df['property_damage'].mode()[0])
df['police_report_available'] = df['police_report_available'].fillna(df['police_report_available'].mode()[0])

In [208]:
o_col = df.select_dtypes(include = ['object'])
num_df = df.select_dtypes(include = ['int64'])

In [209]:
o_col = pd.get_dummies(o_col)

In [210]:

df = pd.concat([num_df, o_col], axis = 1)

In [211]:
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size = 0.2, random_state = 42)

In [212]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': list(range(1, 10, 2)) + [None],
    'min_samples_leaf': range(1, 10, 2),
    'min_samples_split': range(1, 10, 2),
}

In [213]:
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv = 5)

In [214]:
#grid_search.fit(X_train, y_train)

In [215]:
#grid_search.best_estimator_

In [216]:
dt = DecisionTreeClassifier(
                            max_depth = 1,
                            min_samples_leaf= 3, 
                            min_samples_split = 3, 
                            random_state = 42,)

In [217]:
dt.fit(X_train, y_train)
print(f1_score(dt.predict(X_train), y_train))
print(f1_score(dt.predict(X_test), y_test))

0.6539379474940334
0.5769230769230769


In [218]:
rf = RandomForestClassifier(criterion= 'entropy',
                            min_samples_leaf= 1, 
                            min_samples_split = 10, 
                            n_estimators= 82, 
                            random_state = 42,
                            class_weight = 'balanced_subsample',
)
rf.fit(X_train, y_train)
print(f1_score(rf.predict(X_train), y_train))
print(f1_score(rf.predict(X_test), y_test))

0.8809523809523809
0.5631067961165048


In [219]:
c = np.array(rf.feature_importances_)
features = np.array(df.columns)  # X là dataframe train input

# Tạo dataframe để dễ sort
feat_df = pd.DataFrame({'feature': features, 'importance': c})

# Sort giảm dần và in 5 feature quan trọng nhất
top5 = feat_df.sort_values(by='importance', ascending=False).head(5)
print(top5)

                           feature  importance
51  incident_severity_Major Damage    0.173214
10                  property_claim    0.074933
9                     injury_claim    0.061751
11                   vehicle_claim    0.058697
52  incident_severity_Minor Damage    0.054481


In [220]:
c = np.array(dt.feature_importances_)
features = np.array(df.columns)  # X là dataframe train input

# Tạo dataframe để dễ sort
feat_df = pd.DataFrame({'feature': features, 'importance': c})

# Sort giảm dần và in 5 feature quan trọng nhất
top5 = feat_df.sort_values(by='importance', ascending=False).head(5)
print(top5)

                           feature  importance
51  incident_severity_Major Damage         1.0
1                policy_deductable         0.0
0               months_as_customer         0.0
3                    capital-gains         0.0
4                     capital-loss         0.0


In [221]:
df.head()

,months_as_customer,policy_deductable,umbrella_limit,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,...,incident_severity_Total Loss,incident_severity_Trivial Damage,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_Other,authorities_contacted_Police,property_damage_NO,property_damage_YES,police_report_available_NO,police_report_available_YES
0,328,1000,0,53300,0,5,1,1,2,6510,...,False,False,False,False,False,True,False,True,False,True
1,228,2000,5000000,0,0,8,1,0,0,780,...,False,False,False,False,False,True,True,False,True,False
2,134,2000,5000000,35100,0,7,3,2,3,7700,...,False,False,False,False,False,True,True,False,True,False
3,256,2000,6000000,48900,-62400,5,1,1,2,6340,...,False,False,False,False,False,True,True,False,True,False
4,228,1000,6000000,66000,-46000,20,1,0,1,1300,...,False,False,False,False,False,False,True,False,True,False
